In [ ]:

import numpy as np
import pandas as pd
import random
import operator
import pickle
from pathlib import Path
from typing import List, Tuple, Dict, Any
from functools import partial
import traceback

# DEAP imports
from deap import algorithms, base, creator, tools, gp

# ML imports
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def evaluate_auc(clf, x_te, x_tr, y_tr, y_te, labels, weird_probas=False):
    global feat_set_name
    """Compute per‑label and mean ROC‑AUC (Jamendo / MagnaTagATune)."""
    preds_te = clf.predict_proba(x_te)
    preds_tr = clf.predict_proba(x_tr)
    auc_tr, auc_te = [], []
    for i, tag in enumerate(labels):
        p_tr = preds_tr[:, i] if weird_probas else preds_tr[i][:, 1]
        p_te = preds_te[:, i] if weird_probas else preds_te[i][:, 1]
        auc_tr.append(roc_auc_score(y_tr[tag], p_tr))
        auc_te.append(roc_auc_score(y_te[tag], p_te))
    print("Mean AUC  (train):", np.mean(auc_tr))
    print("Mean AUC  (test) :", np.mean(auc_te))

    # build and return DataFrame
    df = pd.DataFrame([{
        'feat_set_name': feat_set_name,
        'auc_tr': np.mean(auc_tr),
        'auc_te': np.mean(auc_te)
    }])

    return df

# 1  Feature‑set definitions  (second cell)
# --- Essentia 23 ‑ low/mid‑level signal descriptors ------------------
E23 = [
    'Danceability', 'Loudness', 'Chords-Changes-Rate', 'Dynamic-Complexity',
    'Zerocrossingrate', 'Chords-Number-Rate', 'Pitch-Salience',
    'Spectral-Centroid', 'Spectral-Complexity', 'Spectral-Decrease',
    'Spectral-Energyband-High', 'Spectral-Energyband-Low',
    'Spectral-Energyband-Middle-High', 'Spectral-Energyband-Middle-Low',
    'Spectral-Entropy', 'Spectral-Flux', 'Spectral-Rolloff',
    'Spectral-Spread', 'Onset-Rate', 'Length', 'BPM', 'Beats-Loud'
]

# --- Mid‑level perceptual 7 ------------------------------------------
ML7 = [
    'Melody', 'Articulation', 'Rhythm Complexity', 'Rhythm Stability',
    'Dissonance', 'Atonality', 'Mode'
]

# --- Symbolic / harmony 32 -------------------------------------------
SYM32 = [
    'Dominants', 'Subdominants', 'sub-sub', 'sub-dom', 'dom-sub',
    'dom-tonic', 'glob-sub', 'glob-dom', 'sub-sub-dom', 'sub-dom-sub',
    'dom-sub-dom', 'sub-dom-tonic', 'dom-tonic-sub', 'dom-sub-sub',
    'sub-sub-sub', 'glob-sub-glob', 'glob-dom-tonic', 'glob-sub-sub',
    'dom-dom', 'glob-glob', 'dom-dom-sub', 'glob-glob-dom',
    'glob-dom-glob', 'glob-glob-sub', 'dom-dom-tonic', 'glob-sub-dom',
    'dom-tonic-dom', 'glob-dom-sub', 'sub-dom-dom', 'dom-dom-dom',
    'glob-dom-dom', 'glob-glob-glob'
]

ALL62 = ML7 + SYM32 + E23      # full perceptual set

all = ['Melody','Articulation','Rhythm Complexity','Rhythm Stability', 'Dissonance', 'Atonality', 'Mode', 
    'Dominants', 'Subdominants', 'sub-sub', 'sub-dom', 'dom-sub', 'dom-tonic', 'glob-sub',  'glob-dom', 
    'sub-sub-dom', 'sub-dom-sub', 'dom-sub-dom', 'sub-dom-tonic', 'dom-tonic-sub', 
    'dom-sub-sub', 'sub-sub-sub', 'glob-sub-glob','glob-dom-tonic', 'glob-sub-sub', 'dom-dom', 'glob-glob',  'dom-dom-sub', 'glob-glob-dom', 'glob-dom-glob', 
    'glob-glob-sub',  'dom-dom-tonic', 'glob-sub-dom',  'dom-tonic-dom',  'glob-dom-sub', 'sub-dom-dom',  'dom-dom-dom','glob-dom-dom', 'glob-glob-glob',  'Danceability','Loudness','Chords-Changes-Rate','Dynamic-Complexity','Zerocrossingrate','Chords-Number-Rate'
    ,'Pitch-Salience','Spectral-Centroid','Spectral-Complexity','Spectral-Decrease','Spectral-Energyband-High',
    'Spectral-Energyband-Low','Spectral-Energyband-Middle-High','Spectral-Energyband-Middle-Low','Spectral-Entropy','Spectral-Flux','Spectral-Rolloff','Spectral-Spread','Onset-Rate','Length','BPM',
    'Beats-Loud']

# 2  Global hyper‑parameters  (third cell)
# ---------------------------------------------------------------------
# Choose feature subset for baseline:
#   feat_set = E23        # -> “signal‑processing only”  (M0 of E1)
#   feat_set = ALL62      # -> full perceptual set       (M0 of E2)
# ---------------------------------------------------------------------
feat_set = E23       # <‑‑ change here for E1 vs E2
if feat_set == ALL62:
    feat_set_name = "ALL62"
elif feat_set == E23:
    feat_set_name = "E23"
# xgb_params_binary = dict(
#     max_depth=3, learning_rate=0.1, n_estimators=70,
#     gamma=7.56, min_child_weight=6,
#     objective='binary:logistic', eval_metric='auc'
# )
# xgb_params_multi = dict(
#     max_depth=2, learning_rate=0.3, objective='multi:softmax',
#     num_class=10, importance_type='weight'
# )
xgb_gpu_binary = dict(
    tree_method='hist', device='cuda',      # use CUDA kernels  :contentReference[oaicite:2]{index=2}
    n_estimators=70,
    max_depth=3, learning_rate=0.1,
    gamma=7.56, min_child_weight=6,
    objective='binary:logistic', eval_metric='auc'
)

xgb_gpu_multi = dict(
    tree_method='hist', device='cuda',
    max_depth=2, learning_rate=0.3,
    objective='multi:softmax', num_class=10,     # GTZAN   :contentReference[oaicite:4]{index=4}
    importance_type='weight'
)

lgb_gpu_params = dict(
    boosting_type='gbdt',
    device='gpu',                # turn on GPU  :contentReference[oaicite:5]{index=5}
    gpu_platform_id=-1,          # -1 -> first platform  :contentReference[oaicite:6]{index=6}
    gpu_device_id=0,             # or pick a specific card
    n_estimators=300,
    max_depth=-1,
    learning_rate=0.05,
    objective='binary',
    metric='auc'
)

RANDOM_STATE = 4
DATA_ROOT = Path('D:/ICASSP1_GPMusic')   # adjust if CSVs live elsewhere


In [ ]:
# Define GP Class

In [ ]:
class AudioFeatureGP:
    """
    Multi-feature Genetic Programming for Audio Feature Engineering
    
    This implementation follows research best practices:
    - Multi-tree approach for multiple feature construction
    - Domain-specific operators for audio/music features
    - Multi-objective fitness with complexity penalties
    - Incremental feature addition strategy
    """
    
    def __init__(self, 
                 population_size: int = 100,
                 generations: int = 50,
                 max_depth: int = 6,
                 crossover_prob: float = 0.8,
                 mutation_prob: float = 0.05,
                 tournament_size: int = 3,
                 complexity_weight: float = 0.01,
                 elite_size: int = 5,
                 random_state: int = 42):
        """
        Initialize GP with research-backed hyperparameters
        
        Args:
            population_size: 50-200 optimal for audio tasks (research: 75-100 for music)
            generations: 25-50 sufficient for convergence
            max_depth: 5-8 optimal balance (research shows 6 works well)
            crossover_prob: 0.8-0.9 optimal (research: 0.8-0.9)
            mutation_prob: 0.02-0.05 for audio (research: lower for feature selection)
            tournament_size: 3-5 consistently outperforms alternatives
            complexity_weight: 0.001-0.1 of accuracy weight
            elite_size: 3-5% of population for preservation
            random_state: For reproducibility
        """
        self.population_size = population_size
        self.generations = generations
        self.max_depth = max_depth
        self.crossover_prob = crossover_prob
        self.mutation_prob = mutation_prob
        self.tournament_size = tournament_size
        self.complexity_weight = complexity_weight
        self.elite_size = elite_size
        self.random_state = random_state

        self.history = []       # will hold per-individual dicts
        self.save_path = None   # filled in by caller before .fit()
        
        # Initialize random state
        random.seed(random_state)
        np.random.seed(random_state)
        
        # Storage for results and features
        self.best_features = []
        self.feature_expressions = []
        self.evolution_log = []
        self.baseline_results = None
        
        # Feature construction tracking
        self.constructed_features_data = None
        self.feature_correlations = {}

    def benchmark_classifiers(self, X_val: np.ndarray, y_val: np.ndarray, 
                            X_test: np.ndarray, y_test: np.ndarray,
                            xgb_params: dict, n_trials: int = 10) -> Dict:
        """
        Benchmark Random Forest vs GPU XGBoost for fitness evaluation speed
        
        Args:
            X_val, y_val: Validation data for testing
            X_test, y_test: Test data for testing  
            xgb_params: Your xgb_gpu_binary parameters
            n_trials: Number of trials to average over
            
        Returns:
            Dictionary with timing results and recommendations
        """
        import time
        
        print("🏁 BENCHMARKING CLASSIFIERS FOR FITNESS EVALUATION")
        print("=" * 60)
        
        # Create dummy feature for testing (single feature like in GP evaluation)
        dummy_feature_val = np.random.randn(X_val.shape[0], 1)
        dummy_feature_test = np.random.randn(X_test.shape[0], 1)
        
        # Test Random Forest
        print("Testing Random Forest...")
        rf_times = []
        rf_aucs_val = []
        rf_aucs_test = []
        
        for trial in range(n_trials):
            start_time = time.time()
            
            clf_rf = RandomForestClassifier(n_estimators=50, random_state=self.random_state, n_jobs=1)
            auc_scores_val = []
            auc_scores_test = []
            
            for i in range(y_val.shape[1]):
                try:
                    clf_rf.fit(dummy_feature_val, y_val[:, i])
                    
                    # Validation AUC
                    y_pred_proba_val = clf_rf.predict_proba(dummy_feature_val)[:, 1]
                    auc_val = roc_auc_score(y_val[:, i], y_pred_proba_val)
                    auc_scores_val.append(auc_val)
                    
                    # Test AUC  
                    y_pred_proba_test = clf_rf.predict_proba(dummy_feature_test)[:, 1]
                    auc_test = roc_auc_score(y_test[:, i], y_pred_proba_test)
                    auc_scores_test.append(auc_test)
                except:
                    auc_scores_val.append(0.5)
                    auc_scores_test.append(0.5)
            
            rf_times.append(time.time() - start_time)
            rf_aucs_val.append(np.mean(auc_scores_val))
            rf_aucs_test.append(np.mean(auc_scores_test))
        
        # Test XGBoost
        print("Testing GPU XGBoost...")
        xgb_times = []
        xgb_aucs_val = []
        xgb_aucs_test = []
        
        for trial in range(n_trials):
            start_time = time.time()
            
            model_xgb = xgb.XGBClassifier(**xgb_params)
            model_xgb.fit(dummy_feature_val, y_val)
            
            # Predictions
            val_pred_proba = model_xgb.predict_proba(dummy_feature_val)
            test_pred_proba = model_xgb.predict_proba(dummy_feature_test)
            
            # Calculate AUCs
            auc_scores_val = []
            auc_scores_test = []
            for i in range(y_val.shape[1]):
                try:
                    proba_val = val_pred_proba[i][:, 1] if len(val_pred_proba[i].shape) > 1 else val_pred_proba[i]
                    proba_test = test_pred_proba[i][:, 1] if len(test_pred_proba[i].shape) > 1 else test_pred_proba[i]
                    
                    auc_val = roc_auc_score(y_val[:, i], proba_val)
                    auc_test = roc_auc_score(y_test[:, i], proba_test)
                    
                    auc_scores_val.append(auc_val)
                    auc_scores_test.append(auc_test)
                except:
                    auc_scores_val.append(0.5)
                    auc_scores_test.append(0.5)
            
            xgb_times.append(time.time() - start_time)
            xgb_aucs_val.append(np.mean(auc_scores_val))
            xgb_aucs_test.append(np.mean(auc_scores_test))
        
        # Calculate statistics
        rf_mean_time = np.mean(rf_times)
        rf_std_time = np.std(rf_times)
        xgb_mean_time = np.mean(xgb_times)
        xgb_std_time = np.std(xgb_times)
        
        speedup = xgb_mean_time / rf_mean_time
        
        print(f"\n📊 BENCHMARK RESULTS ({n_trials} trials):")
        print(f"Random Forest: {rf_mean_time:.3f}±{rf_std_time:.3f}s per evaluation")
        print(f"GPU XGBoost:   {xgb_mean_time:.3f}±{xgb_std_time:.3f}s per evaluation")
        print(f"Speedup factor: {speedup:.2f}x ({'RF faster' if speedup > 1 else 'XGB faster'})")
        
        print(f"\n📈 Performance comparison:")
        print(f"Random Forest Val AUC: {np.mean(rf_aucs_val):.4f}±{np.std(rf_aucs_val):.4f}")
        print(f"GPU XGBoost Val AUC:   {np.mean(xgb_aucs_val):.4f}±{np.std(xgb_aucs_val):.4f}")
        print(f"Random Forest Test AUC: {np.mean(rf_aucs_test):.4f}±{np.std(rf_aucs_test):.4f}")
        print(f"GPU XGBoost Test AUC:   {np.mean(xgb_aucs_test):.4f}±{np.std(xgb_aucs_test):.4f}")
        
        # Recommendation
        if speedup > 1.5:
            recommendation = "Use Random Forest for GP fitness evaluation (significantly faster)"
        elif speedup < 0.67:
            recommendation = "Use GPU XGBoost for GP fitness evaluation (significantly faster)"
        else:
            recommendation = "Both methods have similar speed - choose based on performance quality"
        
        print(f"\n💡 RECOMMENDATION: {recommendation}")
        
        # Estimate total GP time
        total_evaluations = self.population_size * self.generations * 5  # Rough estimate
        rf_total_time = total_evaluations * rf_mean_time / 3600  # Hours
        xgb_total_time = total_evaluations * xgb_mean_time / 3600  # Hours
        
        print(f"\n⏱️  Estimated total GP time (5 features):")
        print(f"With Random Forest: {rf_total_time:.1f} hours")
        print(f"With GPU XGBoost:   {xgb_total_time:.1f} hours")
        print("=" * 60)
        
        return {
            'rf_mean_time': rf_mean_time,
            'rf_std_time': rf_std_time,
            'xgb_mean_time': xgb_mean_time,
            'xgb_std_time': xgb_std_time,
            'speedup': speedup,
            'recommendation': recommendation,
            'rf_val_auc': np.mean(rf_aucs_val),
            'xgb_val_auc': np.mean(xgb_aucs_val),
            'rf_test_auc': np.mean(rf_aucs_test),
            'xgb_test_auc': np.mean(xgb_aucs_test)
        }


    def setup_primitive_set(self, feature_names: List[str]) -> gp.PrimitiveSet:
        """
        Create domain-specific primitive set for audio feature engineering
        
        Based on research, effective audio primitives include:
        - Arithmetic operators for combining spectral features
        - Statistical operations for temporal aggregation
        - Trigonometric functions for periodic analysis
        - Boolean operations for threshold-based features
        """
        # Create primitive set with feature inputs
        pset = gp.PrimitiveSet("MAIN", len(feature_names))
        
        # Rename arguments to feature names for interpretability
        for i, name in enumerate(feature_names):
            pset.renameArguments(**{f'ARG{i}': name.replace(' ', '_').replace('-', '_')})
        
        # Arithmetic operators - essential for combining spectral features
        pset.addPrimitive(operator.add, 2, name="add")
        pset.addPrimitive(operator.sub, 2, name="sub") 
        pset.addPrimitive(operator.mul, 2, name="mul")
        
        # Protected division to avoid division by zero
        def protectedDiv(left, right):
            return left / right if abs(right) > 1e-6 else 1.0
        pset.addPrimitive(protectedDiv, 2, name="div")
        
        # Statistical operators - crucial for temporal aggregation in audio
        def safe_log(x):
            return np.log(abs(x) + 1e-6)
        pset.addPrimitive(safe_log, 1, name="log")
        
        def safe_sqrt(x):
            return np.sqrt(abs(x))
        pset.addPrimitive(safe_sqrt, 1, name="sqrt")
        
        def safe_exp(x):
            return np.exp(np.clip(x, -500, 500))  # Prevent overflow
        pset.addPrimitive(safe_exp, 1, name="exp")

        # Added: Absolute Value
        pset.addPrimitive(np.abs, 1, name="abs")
        
        # Added: Inverse (protected)
        def protected_inv(x):
            return 1.0 / x if abs(x) > 1e-6 else 0.0
        pset.addPrimitive(protected_inv, 1, name="inv")
        
        # Trigonometric functions - valuable for periodic signal analysis
        pset.addPrimitive(np.sin, 1, name="sin")
        pset.addPrimitive(np.cos, 1, name="cos")

        # Added: tan, sinh, cosh, tanh
        def protected_tan(x):
            # tan(x) approaches infinity at odd multiples of pi/2.
            # We'll protect against values very close to these points.
            # A common approach is to return a large but finite number or 0.
            if abs(np.cos(x)) < 1e-6: # Check if cos(x) is near zero
                return 0.0 # Or some large finite number like 1e6 or -1e6
            return np.tan(x)
        pset.addPrimitive(protected_tan, 1, name="tan")
        
        pset.addPrimitive(np.sinh, 1, name="sinh")
        pset.addPrimitive(np.cosh, 1, name="cosh")
        pset.addPrimitive(np.tanh, 1, name="tanh")
        
        # Power functions for nonlinear combinations
        def safe_power(x, y):
            try:
                if x == 0:
                    return 0
                return np.power(abs(x), abs(y) % 3 + 1)  # Limit exponent to prevent overflow
            except:
                return 1.0
        pset.addPrimitive(safe_power, 2, name="pow")

        # Added: square
        def square(x):
            return x * x
        pset.addPrimitive(square, 1, name="square")
        
        # Min/Max operations for feature selection and thresholding
        pset.addPrimitive(min, 2, name="min")
        pset.addPrimitive(max, 2, name="max")

        # Neural Network Activation Functions
        # Sigmoid (Logistic)
        def sigmoid(x):
            # Clip input to prevent overflow in np.exp for large negative x
            return 1 / (1 + np.exp(-np.clip(x, -500, 500)))
        pset.addPrimitive(sigmoid, 1, name="sigmoid")

        # ReLU (Rectified Linear Unit)
        def relu(x):
            return np.maximum(0, x)
        pset.addPrimitive(relu, 1, name="relu")

        # Leaky ReLU
        def leaky_relu(x, alpha=0.01):
            return np.where(x > 0, x, alpha * x)
        # Note: For Leaky ReLU, if you want 'alpha' to be a trainable/fixed hyperparameter,
        # you'd need a different approach than a simple primitive.
        # Here, alpha is fixed to 0.01.
        pset.addPrimitive(leaky_relu, 1, name="lrelu")

        # Swish (Sigmoid-weighted linear unit) - requires multiplication
        # This one is a bit more complex as it's x * sigmoid(x)
        def swish(x):
            return x * (1 / (1 + np.exp(-np.clip(x, -500, 500))))
        pset.addPrimitive(swish, 1, name="swish")
        
        # Conditional operations for threshold-based features
        def if_then_else(condition, output1, output2):
            return output1 if condition > 0 else output2
        pset.addPrimitive(if_then_else, 3, name="if_else")
        
        # Add ephemeral constants
        pset.addEphemeralConstant("rand", lambda: random.uniform(-2, 2))
        
        return pset
        
    def setup_deap_toolbox(self, pset: gp.PrimitiveSet, 
                           X_train: np.ndarray, y_train: np.ndarray,
                          X_val: np.ndarray, y_val: np.ndarray,
                          X_test: np.ndarray = None, y_test: np.ndarray = None,
                          use_xgboost: bool = False, xgb_params: dict = None) -> base.Toolbox:
        """
        Configure DEAP toolbox with fitness function and genetic operators
        
        Uses multi-objective fitness balancing AUC performance with complexity penalty
        Research shows this approach prevents bloat while maintaining performance
        
        Args:
            pset: Primitive set for GP
            X_val, y_val: Validation data for fitness evaluation
            X_test, y_test: Optional test data for monitoring (not used in fitness)
            use_xgboost: Whether to use XGBoost instead of Random Forest
            xgb_params: XGBoost parameters if use_xgboost=True
        """
        # Create fitness and individual classes
        # Maximize AUC, minimize complexity (negative for maximization)
        creator.create("FitnessMulti", base.Fitness, weights=(1.0, -1.0))
        creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMulti)
        
        toolbox = base.Toolbox()
        
        # Expression generation with controlled depth
        toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
        toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        
        # Compilation function
        toolbox.register("compile", gp.compile, pset=pset)
        
        # Store test performance for monitoring (not used in fitness)
        self.test_performance_log = []
        
        # Fitness evaluation with multi-objective approach
        def evaluate_fitness(individual, X_train, y_train, X_val, y_val, X_test=None, y_test=None):
            try:
                # 1) compile GP tree to function
                func = toolbox.compile(expr=individual)

                # 2) produce new feature on val set
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    f_train_raw = np.array([func(*row) for row in X_train]).reshape(-1,1)
                    f_val_raw = np.array([func(*row) for row in X_val]).reshape(-1,1)
                    # --- Perform the non-finite check for both train and val raw data ---
                    if np.any(~np.isfinite(f_train_raw)) or np.any(~np.isfinite(f_val_raw)):
                        raise ValueError("Invalid feature values (NaN or Inf) detected in raw training or validation data.")
                    scaler = StandardScaler().fit(np.vstack((f_train_raw, f_val_raw)))
                    fv_train = scaler.transform(f_train_raw)
                    fv_val = scaler.transform(f_val_raw)

                    # --- CLEAN NaN, Inf, and clip extremes ---
                    fv_train = np.nan_to_num(fv_train, nan=0.0, posinf=1e6, neginf=-1e6)
                    fv_val   = np.nan_to_num(fv_val,   nan=0.0, posinf=1e6, neginf=-1e6)

                    # optional test feature (for monitoring)
                    fv_test_s = None
                    if X_test is not None and y_test is not None:
                        try:
                            ft = np.array([func(*row) for row in X_test]).reshape(-1,1)
                            fv_test_s = scaler.transform(ft)
                        except:
                            fv_test_s = None

                # 3) combine with originals
                X_train_comb = np.hstack([X_train, fv_train])
                X_val_comb   = np.hstack([X_val,   fv_val])
                if fv_test_s is not None:
                    X_test_comb = np.hstack([X_test, fv_test_s])

                # 1) Collapse any one-hot labels into class indices:
                if y_train.ndim > 1:
                    # Multi-class with one-hot: convert to single integer per sample
                    y_train_s = np.argmax(y_train, axis=1)
                    y_val_s   = np.argmax(y_val,   axis=1)
                    y_test_s  = np.argmax(y_test,  axis=1) if y_test is not None else None
                else:
                    # Already 1D labels
                    y_train_s, y_val_s, y_test_s = y_train, y_val, y_test

                # 5) train & score
                if use_xgboost and xgb_params:
                    clf = xgb.XGBClassifier(**xgb_params, missing=0.0)
                else:
                    clf = RandomForestClassifier(
                                n_estimators=50,
                                random_state=self.random_state,
                                n_jobs=1)
                clf.fit(X_train_comb, y_train_s)

                # — compute TRAIN AUC —
                acc_tr = accuracy_score(y_train_s, clf.predict(X_train_comb))
                individual.train_auc = acc_tr

                # 5) score on VALIDATION split
                acc_val = accuracy_score(y_val_s, clf.predict(X_val_comb))
                mean_val_auc = acc_val
                individual.val_auc = acc_val

                # 6) optional TEST monitoring
                if fv_test_s is not None:
                    acc_test = accuracy_score(y_test_s, clf.predict(X_test_comb))
                    individual.test_auc = acc_test

                # — extract GP-feature importances —
                if hasattr(clf, "estimators_"):
                    # Multi-output: average importances across sub-estimators
                    imps    = np.stack([est.feature_importances_ for est in clf.estimators_])
                    avg_imp = imps.mean(axis=0)
                else:
                    # Single-label: direct feature_importances_ array
                    avg_imp = clf.feature_importances_

                # original feature count = X_train.shape[1]
                gp_imp = avg_imp[X_train.shape[1]:]
                individual.gp_importances = gp_imp.tolist()

                # complexity penalty is tree size
                complexity = len(individual)

                return mean_val_auc, complexity

            except Exception as e:
                # 1) Print the exception message
                print(f"[evaluate_fitness] Caught exception: {e}")

                # 2) Print full traceback for debugging
                traceback.print_exc()
                
                # 3) Fallback to worst fitness
                individual.train_auc = 0.0
                individual.val_auc   = 0.0
                individual.test_auc  = 0.0
                individual.gp_importances = []
                return 0.0, len(individual)
        
        toolbox.register("evaluate", evaluate_fitness, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, 
                        X_test=X_test, y_test=y_test)
        
        # Selection and genetic operators based on research recommendations
        toolbox.register("select", tools.selTournament, tournsize=self.tournament_size)
        toolbox.register("mate", gp.cxOnePoint)
        toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
        toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
        
        # Restrict tree depth to prevent bloat
        toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=self.max_depth))
        toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=self.max_depth))
        
        return toolbox
    
    def check_feature_correlation(self, new_feature: np.ndarray, 
                                existing_features: np.ndarray, threshold: float = 0.95) -> bool:
        """
        Check if new feature is too correlated with existing features
        Research shows correlation-based redundancy management prevents duplicate features
        """
        if existing_features.shape[1] == 0:
            return False
            
        correlations = [abs(np.corrcoef(new_feature, existing_features[:, i])[0, 1]) 
                       for i in range(existing_features.shape[1])]
        max_correlation = max(correlations) if correlations else 0
        
        return max_correlation > threshold
    
    def evolve_single_feature(self, feature_idx, X_train: np.ndarray, y_train: np.ndarray,
                            X_val: np.ndarray, y_val: np.ndarray,
                            X_test: np.ndarray, y_test: np.ndarray,
                            existing_features: np.ndarray,
                            feature_names: List[str],
                            use_xgboost: bool = False, 
                            xgb_params: dict = None) -> Tuple[Any, Dict]:
        """
        Evolve a single new feature using GP
        
        Returns best individual and evolution statistics
        """
        print(f"Initializing GP with {len(feature_names)} input features...")
        
        # Setup GP components
        pset = self.setup_primitive_set(feature_names)
        toolbox = self.setup_deap_toolbox(pset, X_train, y_train, X_val, y_val, X_test, y_test, use_xgboost, xgb_params)
        
        # Create initial population
        print(f"Creating initial population of {self.population_size} individuals...")
        population = toolbox.population(n=self.population_size)
        
        # Evaluate initial population
        print("Evaluating initial population...")
        fitnesses = list(map(toolbox.evaluate, population))
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit
            # log it
            self.history.append({
                'phase':  'init',
                'feature_round': feature_idx+1,
                'generation':     0,
                'expression':     str(ind),
                'train_auc':      getattr(ind, 'train_auc', None),
                'val_auc':        getattr(ind, 'val_auc',   None),
                'test_auc':       getattr(ind, 'test_auc',  None),
                'gp_importances': getattr(ind, 'gp_importances', [])
            })
        
        # Initial population statistics
        initial_auc_scores = [ind.fitness.values[0] for ind in population]
        initial_complexities = [ind.fitness.values[1] for ind in population]
        initial_test_aucs = [getattr(ind, 'test_auc', None) for ind in population]
        initial_test_aucs = [auc for auc in initial_test_aucs if auc is not None]
        
        print(f"Initial population stats:")
        print(f"  Val AUC: min={min(initial_auc_scores):.4f}, max={max(initial_auc_scores):.4f}, "
              f"avg={np.mean(initial_auc_scores):.4f}, std={np.std(initial_auc_scores):.4f}")
        if initial_test_aucs:
            print(f"  Test AUC: min={min(initial_test_aucs):.4f}, max={max(initial_test_aucs):.4f}, "
                  f"avg={np.mean(initial_test_aucs):.4f}, std={np.std(initial_test_aucs):.4f}")
        print(f"  Complexity: min={min(initial_complexities):.1f}, max={max(initial_complexities):.1f}, "
              f"avg={np.mean(initial_complexities):.1f}")
        
        # Show best initial individual
        best_initial = tools.selBest(population, 1)[0]
        initial_test_auc = getattr(best_initial, 'test_auc', 'N/A')
        print(f"  Best initial: Val AUC={best_initial.fitness.values[0]:.4f}, "
              f"Test AUC={initial_test_auc if initial_test_auc != 'N/A' else 'N/A':.4f}, "
              f"Complexity={best_initial.fitness.values[1]:.1f}")
        print(f"  Expression: {str(best_initial)}")
        
        # Evolution statistics
        stats_auc = tools.Statistics(lambda ind: ind.fitness.values[0])  # Track validation AUC
        stats_auc.register("avg", np.mean)
        stats_auc.register("max", np.max)
        stats_auc.register("min", np.min)
        stats_auc.register("std", np.std)
        
        stats_complexity = tools.Statistics(lambda ind: ind.fitness.values[1])  # Track complexity
        stats_complexity.register("avg", np.mean)
        stats_complexity.register("max", np.max)
        stats_complexity.register("min", np.min)
        
        # Test AUC statistics (for monitoring)
        def get_test_auc(ind):
            return getattr(ind, 'test_auc', 0.5)  # Default to random if not available
        stats_test = tools.Statistics(get_test_auc)
        stats_test.register("avg", np.mean)
        stats_test.register("max", np.max)
        stats_test.register("min", np.min)
        
        logbook = tools.Logbook()
        logbook.header = ['gen', 'nevals', 'val_avg', 'val_max', 'val_min', 'val_std', 
                         'test_avg', 'test_max', 'test_min', 'comp_avg', 'comp_max', 'comp_min']
        
        print(f"\nStarting evolution for {self.generations} generations...")
        print("Gen | Nevals | Val_avg  | Val_max  | Test_avg | Test_max | Val-Test | Comp_avg | Best_expr")
        print("-" * 100)

        stagnation_counter = 0
        best_fitness_ever = -1.0
        
        # Evolution loop
        for generation in range(self.generations):
            # Select next generation
            offspring = toolbox.select(population, len(population) - self.elite_size)
            offspring = list(map(toolbox.clone, offspring))

            # Apply crossover and mutation
            crossover_count = 0
            mutation_count = 0
            
            # Apply crossover and mutation
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < self.crossover_prob:
                    toolbox.mate(child1, child2)
                    del child1.fitness.values
                    del child2.fitness.values
                    crossover_count += 2
            
            for mutant in offspring:
                if random.random() < self.mutation_prob:
                    toolbox.mutate(mutant)
                    del mutant.fitness.values
                    mutation_count += 1
            
            # Evaluate new individuals
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = map(toolbox.evaluate, invalid_ind)
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit

                # *** ADD THIS BLOCK: log each re-evaluated individual ***
                self.history.append({
                    'phase':         'eval',
                    'feature_round': feature_idx+1,
                    'last_feature_exp': self.feature_expressions[-1] if self.feature_expressions else None,
                    'generation':    generation+1,
                    'expression':    str(ind),
                    'train_auc':     getattr(ind, 'train_auc', None),
                    'val_auc':       getattr(ind, 'val_auc',   None),
                    'test_auc':      getattr(ind, 'test_auc',  None),
                    'gp_importances':getattr(ind, 'gp_importances', [])
                })
            
            # Elite preservation - keep best individuals
            elite = tools.selBest(population, self.elite_size)
            population[:] = offspring + elite
            
            # Record statistics
            record_val = stats_auc.compile(population)
            record_complexity = stats_complexity.compile(population)
            record_test = stats_test.compile(population)
            
            logbook.record(gen=generation, nevals=len(invalid_ind), 
                          val_avg=record_val['avg'], val_max=record_val['max'], 
                          val_min=record_val['min'], val_std=record_val['std'],
                          test_avg=record_test['avg'], test_max=record_test['max'],
                          test_min=record_test['min'],
                          comp_avg=record_complexity['avg'], comp_max=record_complexity['max'],
                          comp_min=record_complexity['min'])
            
            # Get current best individual
            current_best = tools.selBest(population, 1)[0]
            current_best_val_auc = current_best.fitness.values[0]
            current_best_test_auc = getattr(current_best, 'test_auc', None)
            
            # Check for improvement
            if current_best_val_auc > best_fitness_ever:
                best_fitness_ever = current_best_val_auc
                stagnation_counter = 0
                improvement_marker = " ✓"
            else:
                stagnation_counter += 1
                improvement_marker = ""
            
            # Calculate validation-test gap for overfitting monitoring
            val_test_gap = record_val['max'] - record_test['max'] if record_test['max'] > 0 else 0
            
            # Progress display every generation
            print(f"{generation:3d} | {len(invalid_ind):6d} | {record_val['avg']:8.4f} | "
                  f"{record_val['max']:8.4f} | {record_test['avg']:8.4f} | {record_test['max']:8.4f} | "
                  f"{val_test_gap:8.4f} | {record_complexity['avg']:8.1f} | "
                  f"{str(current_best)[:20]}...{improvement_marker}")

            df_hist = pd.DataFrame(self.history)
            df_hist.to_csv(self.save_path / 'dark_gp_individual_history.csv', index=False)
            df_hist.to_pickle(self.save_path / 'dark_gp_individual_history.pkl')
            
            # Detailed stats every 10 generations
            if (generation + 1) % 10 == 0:
                print(f"\n--- Generation {generation + 1} Detailed Stats ---")
                print(f"Population diversity: {len(set(str(ind) for ind in population))} unique individuals")
                print(f"Genetic operations: {crossover_count} crossovers, {mutation_count} mutations")
                print(f"Stagnation count: {stagnation_counter}")
                print(f"Current best expression: {str(current_best)}")
                # Build a safe string for test AUC
                test_auc_str = (f"{current_best_test_auc:.4f}"
                                if current_best_test_auc is not None
                                else "N/A")
                print(
                    f"Current best fitness: Val AUC={current_best.fitness.values[0]:.4f}, "
                    f"Test AUC={test_auc_str}, "
                    f"Complexity={current_best.fitness.values[1]:.1f}"
                )
                
                # Show top 3 individuals with test performance
                top_3 = tools.selBest(population, 3)
                print("Top 3 individuals:")
                for i, ind in enumerate(top_3):
                    test_auc = getattr(ind, 'test_auc', None)
                    # Pre‑compute per‑individual test‑AUC string
                    indiv_test_str = (f"{test_auc:.4f}"
                                      if test_auc is not None
                                      else "N/A")
                    print(
                        f"  {i+1}. Val AUC={ind.fitness.values[0]:.4f}, "
                        f"Test AUC={indiv_test_str}, "
                        f"Complexity={ind.fitness.values[1]:.1f}, "
                        f"Expr={str(ind)[:40]}..."
                    )
                
                # Overfitting warning
                if val_test_gap > 0.05:
                    print(f"⚠️  WARNING: Large val-test gap ({val_test_gap:.4f}) - possible overfitting!")
                print()
            
            # Early stopping conditions
            if generation > 10 and stagnation_counter >= 15:
                print(f"\nEarly stopping at generation {generation} (no improvement for 15 generations)")
                break
                
            if generation > 5 and len(logbook) > 5:
                recent_max = [logbook[i]['val_max'] for i in range(-5, 0)]
                if max(recent_max) - min(recent_max) < 0.0001:
                    print(f"\nEarly stopping at generation {generation} (convergence detected)")
                    break
        
        # Final evolution summary
        print(f"\n--- Evolution Complete ---")
        print(f"Generations run: {generation + 1}")
        print(f"Final population diversity: {len(set(str(ind) for ind in population))}")
        
        # Return best individual and evolution info
        best_individual = tools.selBest(population, 1)[0]
        best_test_auc = getattr(best_individual, 'test_auc', None)
        
        print(f"Best individual found:")
        print(f"  Val AUC: {best_individual.fitness.values[0]:.4f}")
        best_test_auc_str = f"{best_test_auc:.4f}" if best_test_auc is not None else "N/A"
        print(f"Test AUC: {best_test_auc_str}")
        print(f"  Complexity: {best_individual.fitness.values[1]:.1f}")
        print(f"  Expression: {str(best_individual)}")
        
        evolution_info = {
            'generations_run': generation + 1,
            'final_fitness': best_individual.fitness.values,
            'final_test_auc': best_test_auc,
            'population_diversity': len(set(str(ind) for ind in population)),
            'logbook': logbook,
            'stagnation_counter': stagnation_counter,
            'improvement_generations': [i for i, record in enumerate(logbook) 
                                      if i == 0 or record['val_max'] > logbook[i-1]['val_max']]
        }
        
        return best_individual, evolution_info
    
    def construct_feature_from_individual(self, individual: Any, 
                                        X: np.ndarray, feature_names: List[str]) -> np.ndarray:
        """
        Construct actual feature values from GP individual
        """
        pset = self.setup_primitive_set(feature_names)
        func = gp.compile(individual, pset)
        
        try:
            feature_values = np.array([func(*row) for row in X])
            
            # Handle invalid values
            if np.any(~np.isfinite(feature_values)):
                feature_values = np.nan_to_num(feature_values, nan=0.0, posinf=1.0, neginf=-1.0)
            
            return feature_values
        except Exception as e:
            print(f"Error constructing feature: {e}")
            return np.zeros(X.shape[0])
    
    def fit(self, X_train: np.ndarray, y_train: np.ndarray,
            X_val: np.ndarray, y_val: np.ndarray,
            X_test: np.ndarray, y_test: np.ndarray,
            feature_names: List[str],
            max_features: int = 5,
            baseline_results: pd.DataFrame = None,
            use_xgboost: bool = False,
            xgb_params: dict = None) -> Dict:
        """
        Main fitting method implementing iterative feature construction
        
        Args:
            X_train, y_train: Training data
            X_val, y_val: Validation data for fitness evaluation
            X_test, y_test: Test data for final evaluation and monitoring
            feature_names: Names of original features
            max_features: Maximum number of GP features to construct
            baseline_results: Baseline model results for comparison
            use_xgboost: Whether to use XGBoost instead of Random Forest for fitness
            xgb_params: XGBoost parameters if use_xgboost=True
            
        Returns:
            Dictionary with comprehensive results
        """
        print("=" * 80)
        print("🎵 STARTING MULTI-FEATURE GP EVOLUTION FOR AUDIO FEATURES 🎵")
        print("=" * 80)
        print(f"📊 Dataset Info:")
        print(f"   Training samples: {X_train.shape[0]}")
        print(f"   Validation samples: {X_val.shape[0]}")
        print(f"   Test samples: {X_test.shape[0]}")
        print(f"   Original features: {X_train.shape[1]}")
        # print(f"   Target labels: {y_train.shape[1]}")
        print(f"🔧 GP Parameters:")
        print(f"   Population size: {self.population_size}")
        print(f"   Max generations: {self.generations}")
        print(f"   Max tree depth: {self.max_depth}")
        print(f"   Crossover prob: {self.crossover_prob}")
        print(f"   Mutation prob: {self.mutation_prob}")
        print(f"   Complexity weight: {self.complexity_weight}")
        print(f"   Target GP features: {max_features}")
        print(f"   Fitness classifier: {'GPU XGBoost' if use_xgboost else 'Random Forest'}")
        
        if baseline_results is not None:
            baseline_train_auc = baseline_results['acc_tr'].iloc[0]
            baseline_test_auc = baseline_results['acc_te'].iloc[0]
            print(f"📈 Baseline Performance:")
            print(f"   Train AUC: {baseline_train_auc:.4f}")
            print(f"   Test AUC: {baseline_test_auc:.4f}")
            print(f"   Target: Beat {baseline_test_auc:.4f} on test set")
        
        print("=" * 80)
        
        self.baseline_results = baseline_results
        
        # Initialize feature storage
        gp_features_train = np.empty((X_train.shape[0], 0))
        gp_features_val = np.empty((X_val.shape[0], 0))
        gp_features_test = np.empty((X_test.shape[0], 0))
        
        all_results = []
        feature_construction_log = []
        
        # Iterative feature construction
        for feature_idx in range(max_features):
            print(f"\n🧬 EVOLVING FEATURE {feature_idx + 1}/{max_features}")
            print("=" * 60)
            
            # Track time for this feature
            import time
            start_time = time.time()
            
            # Evolve new feature
            best_individual, evolution_info = self.evolve_single_feature(
                feature_idx, X_train, y_train, X_val, y_val, X_test, y_test,
                gp_features_val, feature_names, use_xgboost, xgb_params
            )
            
            evolution_time = time.time() - start_time
            print(f"⏱️  Evolution completed in {evolution_time:.1f} seconds")
            
            # Construct feature values
            print("🔨 Constructing feature values for all datasets...")
            new_feature_train = self.construct_feature_from_individual(
                best_individual, X_train, feature_names)
            new_feature_val = self.construct_feature_from_individual(
                best_individual, X_val, feature_names)
            new_feature_test = self.construct_feature_from_individual(
                best_individual, X_test, feature_names)
            
            # Feature quality checks
            print("🔍 Feature quality checks:")
            print(f"   Train: min={np.min(new_feature_train):.3f}, max={np.max(new_feature_train):.3f}, "
                  f"mean={np.mean(new_feature_train):.3f}, std={np.std(new_feature_train):.3f}")
            print(f"   NaN/Inf values: {np.sum(~np.isfinite(new_feature_train))}")
            
            # Check correlation with existing features
            if self.check_feature_correlation(new_feature_val, gp_features_val):
                print(f"❌ Feature {feature_idx + 1} too correlated with existing features (>0.95), skipping...")
                feature_construction_log.append({
                    'feature_idx': feature_idx + 1,
                    'status': 'rejected_correlation',
                    'expression': str(best_individual),
                    'fitness': best_individual.fitness.values,
                    'evolution_time': evolution_time
                })
                continue
            
            # Add new feature
            gp_features_train = np.column_stack([gp_features_train, new_feature_train])
            gp_features_val = np.column_stack([gp_features_val, new_feature_val])
            gp_features_test = np.column_stack([gp_features_test, new_feature_test])
            
            # Store feature expression for interpretability
            feature_expression = str(best_individual)
            self.feature_expressions.append(feature_expression)
            print(f"✅ Feature {feature_idx + 1} ACCEPTED!")
            print(f"   Expression: {feature_expression}")
            
            # Evaluate current feature set
            print(f"📊 Evaluating feature set with {len(self.feature_expressions)} GP features...")
            results = self.evaluate_feature_set(
                X_train, y_train, X_val, y_val, X_test, y_test,
                gp_features_train, gp_features_val, gp_features_test,
                len(self.feature_expressions), evolution_info
            )
            
            all_results.append(results)
            
            # Performance summary
            print(f"🎯 Performance Results:")
            print(f"   Training AUC:   {results['train_auc']:.4f} (Δ{results['train_diff']:+.4f})")
            print(f"   Validation AUC: {results['val_auc']:.4f} (Δ{results['val_diff']:+.4f})")
            print(f"   Test AUC:       {results['test_auc']:.4f} (Δ{results['test_diff']:+.4f})")
            
            if results['test_diff'] > 0:
                print(f"   🎉 IMPROVEMENT! Test AUC increased by {results['test_diff']:.4f}")
            elif results['test_diff'] > -0.01:
                print(f"   😐 Marginal change in test AUC ({results['test_diff']:+.4f})")
            else:
                print(f"   😞 Test AUC decreased by {abs(results['test_diff']):.4f}")
            
            feature_construction_log.append({
                'feature_idx': feature_idx + 1,
                'status': 'accepted',
                'expression': feature_expression,
                'fitness': best_individual.fitness.values,
                'evolution_time': evolution_time,
                'results': results
            })
            
            # Performance tracking across features
            if len(all_results) >= 2:
                val_trend = results['val_auc'] - all_results[-2]['val_auc']
                test_trend = results['test_auc'] - all_results[-2]['test_auc']
                print(f"   📈 Trends: Val AUC {val_trend:+.4f}, Test AUC {test_trend:+.4f}")
            
            # Early stopping conditions
            if len(all_results) >= 3:
                recent_val_aucs = [r['val_auc'] for r in all_results[-3:]]
                val_improvement = max(recent_val_aucs) - min(recent_val_aucs)
                
                if val_improvement < 0.001:
                    print(f"\n🛑 EARLY STOPPING: No significant validation improvement in last 3 features")
                    print(f"   (improvement < 0.001: {val_improvement:.6f})")
                    break
            
            # Check if we're overfitting
            if results['train_auc'] - results['test_auc'] > 0.1:
                print(f"⚠️  WARNING: Large train-test gap ({results['train_auc'] - results['test_auc']:.4f})")
                print(f"   Consider reducing complexity or stopping feature construction")
            
            print("=" * 60)
        
        # Store final results
        self.constructed_features_data = {
            'train': gp_features_train,
            'val': gp_features_val, 
            'test': gp_features_test
        }
        
        # Final summary
        print(f"\n🏁 GP FEATURE CONSTRUCTION COMPLETE!")
        print("=" * 80)
        print(f"📈 Summary:")
        print(f"   Features constructed: {len(self.feature_expressions)}")
        print(f"   Features attempted: {len(feature_construction_log)}")
        
        if all_results:
            best_iteration = max(all_results, key=lambda x: x['val_auc'])
            final_iteration = all_results[-1]
            
            print(f"🥇 Best Performance (Feature {best_iteration['num_gp_features']}):")
            print(f"   Validation AUC: {best_iteration['val_auc']:.4f} (Δ{best_iteration['val_diff']:+.4f})")
            print(f"   Test AUC: {best_iteration['test_auc']:.4f} (Δ{best_iteration['test_diff']:+.4f})")
            
            print(f"📊 Final Performance:")
            print(f"   Validation AUC: {final_iteration['val_auc']:.4f} (Δ{final_iteration['val_diff']:+.4f})")
            print(f"   Test AUC: {final_iteration['test_auc']:.4f} (Δ{final_iteration['test_diff']:+.4f})")
            
            if baseline_results is not None:
                if final_iteration['test_diff'] > 0:
                    print(f"   🎉 SUCCESS! Improved test AUC by {final_iteration['test_diff']:.4f}")
                else:
                    print(f"   😞 No improvement over baseline")
        
        print(f"🧬 Constructed Features:")
        for i, expr in enumerate(self.feature_expressions):
            print(f"   GP_Feature_{i+1}: {expr}")
        
        final_results = {
            'all_iterations': all_results,
            'best_iteration': max(all_results, key=lambda x: x['val_auc']) if all_results else None,
            'feature_expressions': self.feature_expressions,
            'total_features_constructed': len(self.feature_expressions),
            'construction_log': feature_construction_log
        }
        
        print("=" * 80)
        
        return final_results
    
    def evaluate_feature_set(self, X_train: np.ndarray, y_train: np.ndarray,
                           X_val: np.ndarray, y_val: np.ndarray,
                           X_test: np.ndarray, y_test: np.ndarray,
                           gp_features_train: np.ndarray, gp_features_val: np.ndarray,
                           gp_features_test: np.ndarray,
                           num_gp_features: int, evolution_info: Dict) -> Dict:
        """
        Evaluate current feature set with comprehensive metrics
        """
        # Combine original and GP features
        X_train_combined = np.column_stack([X_train, gp_features_train])
        X_val_combined = np.column_stack([X_val, gp_features_val])
        X_test_combined = np.column_stack([X_test, gp_features_test])

        # --- Concatenate, fit scaler, and transform both arrays ---
        scaler = StandardScaler().fit(np.vstack((X_train_combined, X_val_combined)))
        X_train_combined_s = scaler.transform(X_train_combined)
        X_val_combined_s = scaler.transform(X_val_combined)
        X_test_combined_s = scaler.transform(X_test_combined)
        
        # Train model with combined features
        model = xgb.XGBClassifier(**xgb_gpu_multi)
        
        
        model.fit(X_train_combined, y_train)
        
        train_auc = accuracy_score(y_train, model.predict(X_train_combined_s))
        val_auc = accuracy_score(y_val, model.predict(X_val_combined_s))
        test_auc = accuracy_score(y_test, model.predict(X_test_combined_s))
        
        # Calculate improvements over baseline
        train_diff = test_diff = val_diff = 0.0
        if self.baseline_results is not None:
            baseline_train_auc = self.baseline_results['acc_tr'].iloc[0]
            baseline_test_auc = self.baseline_results['acc_te'].iloc[0]
            train_diff = train_auc - baseline_train_auc
            test_diff = test_auc - baseline_test_auc
            val_diff = val_auc - baseline_test_auc  # Compare val to baseline test
        
        results = {
            'num_gp_features': num_gp_features,
            'train_auc': train_auc,
            'val_auc': val_auc,
            'test_auc': test_auc,
            'train_diff': train_diff,
            'val_diff': val_diff,
            'test_diff': test_diff,
            'generations_run': evolution_info['generations_run'],
            'final_fitness': evolution_info['final_fitness'],
            'population_diversity': evolution_info['population_diversity'],
            'feature_expressions': self.feature_expressions.copy()
        }
        
        return results
    
    def save_results(self, results: Dict, save_path: Path):
        """Save comprehensive results to disk"""
        # Convert results to DataFrame for easier analysis
        iterations_data = []
        for iteration in results['all_iterations']:
            iterations_data.append({
                'num_gp_features': iteration['num_gp_features'],
                'train_auc': iteration['train_auc'],
                'val_auc': iteration['val_auc'], 
                'test_auc': iteration['test_auc'],
                'train_diff': iteration['train_diff'],
                'val_diff': iteration['val_diff'],
                'test_diff': iteration['test_diff'],
                'generations_run': iteration['generations_run'],
                'population_diversity': iteration['population_diversity'],
                'feature_expression': iteration['feature_expressions'][-1] if iteration['feature_expressions'] else ""
            })
        
        df_results = pd.DataFrame(iterations_data)
        
        # Save results
        df_results.to_csv(save_path / 'dark_gp_feature_evolution_results.csv', index=False)
        df_results.to_pickle(save_path / 'dark_gp_feature_evolution_results.pkl')
        
        # Save complete results object
        with open(save_path / 'dark_gp_complete_results.pkl', 'wb') as f:
            pickle.dump(results, f)
        
        # Save constructed features
        if self.constructed_features_data:
            np.savez(save_path / 'dark_gp_constructed_features.npz',
                    train=self.constructed_features_data['train'],
                    val=self.constructed_features_data['val'],
                    test=self.constructed_features_data['test'])
        
        print(f"Results saved to {save_path}")

In [ ]:
# Example usage with your data
def run_gp_feature_engineering(DATA_ROOT, feat_set, xgb_gpu_binary, benchmark_first: bool = True):
    """
    Complete pipeline for GP feature engineering on your Jamendo dataset
    
    Args:
        DATA_ROOT: Path to your data
        feat_set: List of feature names
        xgb_gpu_binary: Your XGBoost GPU parameters
        benchmark_first: Whether to run classifier benchmark before GP evolution
    """
    
    # ---------------------- GTZAN (single‑label 10 genres) ----------------
    gtzan_csv = DATA_ROOT / 'GTZAN/Ref_Perceptual_features/perceptual_features.csv'
    df = pd.read_csv(gtzan_csv).drop(columns=['Track'])

    # rename to match your other splits
    df.rename(columns={'dom': 'Dominants', 'sub': 'Subdominants'}, inplace=True)

    X = df[feat_set]
    y_onehot = df.drop(columns=all)        # the 10 one‑hot genre cols
    genre_labels = y_onehot.columns.tolist()

    # convert one‑hot → integer labels
    y_int = y_onehot.values.argmax(axis=1)

    # 1) Split off test (20%)
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y_int,
        train_size=0.8,
        random_state=4
    )

    # 2) From the 80% “temp”, split out validation (20% of temp → 16% overall)
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp,
        test_size=0.2,
        # stratify=y_temp,
        random_state=RANDOM_STATE
    )

    # Optional scaling (recommended)
    scaler = StandardScaler().fit(X_temp)
    x_tr_scaled = scaler.transform(X_train)
    x_val_scaled = scaler.transform(X_val)
    x_te_scaled = scaler.transform(X_test)
    
    # Split training data for GP (use validation split for GP fitness evaluation)
    x_train_gp = x_tr_scaled
    y_train_gp = y_train
    x_val_gp = x_val_scaled
    y_val_gp = y_val
    x_test_gp = x_te_scaled
    y_test_gp = y_test

    gtzan_xgb = xgb.XGBClassifier(**xgb_gpu_multi).fit(x_train_gp,y_train_gp)
    print("XGB train acc:", accuracy_score(y_train_gp, gtzan_xgb.predict(x_train_gp)))
    print("XGB test  acc:", accuracy_score(y_test_gp, gtzan_xgb.predict(x_test_gp)))

    # Load baseline results for comparison
    try:
        df_GTZAN_xgb = pd.read_pickle(DATA_ROOT / f'GTZAN/Ref_Perceptual_features/M0_results_xgb_{feat_set_name}.pkl')
        print(f"Loaded baseline results: Train AUC = {df_GTZAN_xgb['acc_tr'].iloc[0]:.4f}, "
              f"Test AUC = {df_GTZAN_xgb['acc_te'].iloc[0]:.4f}")
    except:
        print("Baseline results not found, will run without comparison")
        df_GTZAN_xgb = None

    # Initialize GP
    gp_engineer = AudioFeatureGP(
        population_size=100,    # Research-backed for music tasks
        generations=50,         # Sufficient for convergence
        max_depth=6,           # Optimal balance
        crossover_prob=0.8,    # Research optimum
        mutation_prob=0.1,    # Good for feature construction
        complexity_weight=0.01, # Prevent bloat
        random_state=42
    )

    # Optional: Benchmark classifiers first
    use_xgboost = True # False # Set to False if benchmark_first is true
    if benchmark_first:
        print("\n" + "="*80)
        print("🏁 BENCHMARKING CLASSIFIERS FIRST...")
        print("="*80)
        
        benchmark_results = gp_engineer.benchmark_classifiers(
            x_val_gp, y_val_gp, x_test_gp, y_test_gp, xgb_gpu_binary, n_trials=5
        )
        
        # Use the faster method
        use_xgboost = benchmark_results['speedup'] < 1.0  # XGB faster if speedup < 1
        classifier_name = "GPU XGBoost" if use_xgboost else "Random Forest"
        print(f"\n🚀 Using {classifier_name} for GP fitness evaluation")
        
        if use_xgboost:
            print("⚠️  Note: Using XGBoost may be slower but potentially more accurate")
        else:
            print("⚡ Using Random Forest for faster evolution")

    # Run GP feature engineering
    print(f"\n{'='*80}")
    print("🧬 STARTING GP FEATURE ENGINEERING")
    print(f"Classifier: {'GPU XGBoost' if use_xgboost else 'Random Forest'}")
    print(f"{'='*80}")
    
    gp_engineer.save_path = DATA_ROOT / f'GTZAN/Ref_Perceptual_features/dark_{feat_set_name}_gensize500'
    results = gp_engineer.fit(
        X_train=x_train_gp,
        y_train=y_train_gp,
        X_val=x_val_gp,
        y_val=y_val_gp,
        X_test=x_test_gp,
        y_test=y_test_gp,
        feature_names=feat_set,
        max_features=5,  # Construct up to 5 GP features
        baseline_results=df_GTZAN_xgb,
        use_xgboost=use_xgboost,
        xgb_params=xgb_gpu_multi if use_xgboost else None
    )

    # Save results
    gp_engineer.save_results(results, gp_engineer.save_path)
    
    print("\n" + "="*80)
    print("🎉 GP FEATURE ENGINEERING COMPLETE!")
    print("="*80)
    print(f"Best validation AUC improvement: {results['best_iteration']['val_diff']:.4f}")
    print(f"Best test AUC improvement: {results['best_iteration']['test_diff']:.4f}")
    print(f"Features constructed: {results['total_features_constructed']}")
    
    if benchmark_first:
        print(f"\nBenchmark summary:")
        print(f"  Random Forest: {benchmark_results['rf_mean_time']:.3f}s per evaluation")
        print(f"  GPU XGBoost:   {benchmark_results['xgb_mean_time']:.3f}s per evaluation")
        print(f"  Used: {classifier_name}")
    
    return gp_engineer, results

In [ ]:
# Run the GP feature engineering
gp_engineer, results = run_gp_feature_engineering(DATA_ROOT, feat_set, xgb_gpu_binary, benchmark_first=False)